# adding control points and dv values in .cfg file for adjoint desing optimization

In [1]:
import os #library imported in order to read file in a specific location/folder
from csv import reader         # for reading csv files
from csv import writer         # for writing csv files
import numpy as np             # for mathematical operations
import pandas as pd            # for reading csv files
import shutil                  # for moving file to other directories
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove

In [2]:
#from __future__ import print_function
#from optparse import OptionParser
#from numpy import *

In [3]:
def AddingAdjointOptions():
    Directory=input("enter directory where .cfg file is:")
    ConfigFile=input("enter name .cfg file (it must include .cfg termination):")
    DIM=input("Enter dimension of your problem: 2 or 3:")
    while True:
        try:
            n_boxes=input("Enter Numbers of FFD-Boxes that you want to use: ")
            if int(n_boxes)>0:
                break;
            else:
                print("enter a valid number of FFD-Boxes (an integer bigger than zero)")
        except ValueError:
            print("Invalid")
            continue
    list =""
    Count=0
    for i in range(0,int(n_boxes)):
        FFD_ID=input("Enter FFD BOX ID of the BOX number "+str(i+1)+" (Name of the ffd box,for example: BOX_1):")
        while True:
            try:
                IDegree=input("Enter i degree of the FFD box number "+str(i+1)+" :")
                if int(IDegree)>0:
                    break;
                else:
                    print("enter a valid i degree bigger than 0")
            except ValueError:
                print("Invalid")
                continue
        while True:
            try:
                JDegree=input("Enter j degree of the FFD box number "+str(i+1)+" :")
                if int(JDegree)>0:
                    break;
                else:
                    print("enter a valid j degree bigger than 0")
            except ValueError:
                print("Invalid")
                continue
        if int(DIM)==3:
            while True:
                try:
                    KDegree=input("Enter k degree of the FFD box number "+str(i+1)+" :")
                    if int(KDegree)>0:
                        break;
                    else:
                        print("enter a valid k degree bigger than 0")
                except ValueError:
                    print("Invalid")
                    continue 
        DEF_X=input("Deformation in X direction for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES:")
        DEF_Y=input("Deformation in Y direction for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES:")
        if (int(DIM)==3):
            DEF_Z=input("Deformation in Z direction for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES:")
        BORDERS_X = input("Do you want to include borders in X for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES: ")
        BORDERS_Y = input("Do you want to include borders in Y for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES: ")
        if (int(DIM)==3):
            BORDERS_Z = input("Do you want to include borders in Z for FFD_BOX number "+str(i+1)+" ?: 0=NO, 1=YES:")
        if (int(BORDERS_X)==0):
            border_x=1;
        else:
            border_x=0
        if (int(BORDERS_Y)==0):
            border_y=1;
        else:
            border_y=0
        if (int(DIM)==3):
            if (int(BORDERS_Z)==0):
                border_z=1;
            else:
                border_z=0
        if(int(DIM)==2):
            if (int(DEF_X)==1):
                for j in range(border_y,int(JDegree)+1-border_y):
                    for i in range(border_x, int(IDegree)+1-border_x):
                        list = list + "( "+FFD_ID+", "+str(i)+", "+str(j)+", 1.0, 0.0); "
                        Count=Count+1
            if (int(DEF_Y)==1):
                for j in range(border_y,int(JDegree)+1-border_y):
                    for i in range(border_x,int(IDegree)+1-border_x):
                        list = list + "( "+FFD_ID+", "+str(i)+", "+str(j)+", 0.0, 1.0); "
                        Count=Count+1
        if(int(DIM)==3):
            if (int(DEF_X)==1):
                for k in range(border_z, int(KDegree)+1-border_z):
                    for j in range(border_y, int(JDegree)+1-border_y):
                        for i in range(border_x,int(IDegree)+1-border_x):
                            list = list + "( "+FFD_ID+", "+str(i)+", "+str(j)+", "+str(k)+", 1.0, 0.0, 0.0 ); "
                            Count=Count+1
            if (int(DEF_Y)==1):
                for k in range(border_z, int(KDegree)+1-border_z):
                    for j in range(border_y, int(JDegree)+1-border_y):
                        for i in range(border_x, int(IDegree)+1-border_x):
                            list = list + "( "+FFD_ID+", "+str(i)+", "+str(j)+", "+str(k)+", 0.0, 1.0, 0.0 ); "
                            Count=Count+1
            if (int(DEF_Z)==1):
                for k in range(border_z, int(KDegree)+1-border_z):
                    for j in range(border_y, int(JDegree)+1-border_y):
                        for i in range(border_x, int(IDegree)+1-border_x):
                            list = list + "( "+FFD_ID+", "+str(i)+", "+str(j)+", "+str(k)+", 0.0, 0.0, 1.0 ); "
                            Count=Count+1
        print(Count)
    list=list[:len(list)-2]
    with open(Directory+"/"+ConfigFile, 'r') as file:
        lines = file.readlines()
        number_of_lines = len(lines)
        print(number_of_lines)
    for i in range(number_of_lines):
        check=lines[i].split("=")[0]
        if(check == "DV_KIND"):
            lines[i] = "DV_KIND=";
            for j in range(Count):
                if int(DIM)==2:
                    lines[i] = lines[i]+" "+" FFD_CONTROL_POINT_2D,"
                if int(DIM)==3:
                    lines[i] = lines[i]+" "+" FFD_CONTROL_POINT,"
            lines[i]=lines[i][:len(lines[i])-1]+'\n'
        if(check == "DV_PARAM"):
            #print("FIRST LIST: "+list)
            lines[i] = str(check) + "= "+ str(list)+'\n'
            #print("LINE IN CFG: "+lines[i])
    with open(Directory+"/"+ConfigFile,'w') as file: #.split(".")[0]+"_1.cfg"
        file.writelines(lines)
    file.close()
    #print("THRID LINE: "+list)
    #print(type(list))
    #print(ConfigFile)   

In [4]:
AddingAdjointOptions()

enter directory where .cfg file is: /home/cristopher/Desktop/Documentation/Simulations/SU2_TUTORIAL_TRANSPORT_PROPERTIES/kenics_mixer_optimization_two_boxes_without_edges_double_points_45_4/optimization
enter name .cfg file (it must include .cfg termination): kenics_mixer_tutorial.cfg
Enter dimension of your problem: 2 or 3: 3
Enter Numbers of FFD-Boxes that you want to use:  2
Enter FFD BOX ID of the BOX number 1 (Name of the ffd box,for example: BOX_1): BOX_BLADE_2
Enter i degree of the FFD box number 1 : 20
Enter j degree of the FFD box number 1 : 20
Enter k degree of the FFD box number 1 : 20
Deformation in X direction for FFD_BOX number 1 ?: 0=NO, 1=YES: 1
Deformation in Y direction for FFD_BOX number 1 ?: 0=NO, 1=YES: 0
Deformation in Z direction for FFD_BOX number 1 ?: 0=NO, 1=YES: 0
Do you want to include borders in X for FFD_BOX number 1 ?: 0=NO, 1=YES:  0
Do you want to include borders in Y for FFD_BOX number 1 ?: 0=NO, 1=YES:  0
Do you want to include borders in Z for FFD_BO

6859


Enter FFD BOX ID of the BOX number 2 (Name of the ffd box,for example: BOX_1): BOX_BLADE_3
Enter i degree of the FFD box number 2 : 20
Enter j degree of the FFD box number 2 : 20
Enter k degree of the FFD box number 2 : 20
Deformation in X direction for FFD_BOX number 2 ?: 0=NO, 1=YES: 0
Deformation in Y direction for FFD_BOX number 2 ?: 0=NO, 1=YES: 1
Deformation in Z direction for FFD_BOX number 2 ?: 0=NO, 1=YES: 0
Do you want to include borders in X for FFD_BOX number 2 ?: 0=NO, 1=YES:  0
Do you want to include borders in Y for FFD_BOX number 2 ?: 0=NO, 1=YES:  0
Do you want to include borders in Z for FFD_BOX number 2 ?: 0=NO, 1=YES: 0


13718
261
